In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential



In [2]:
data_dir = pathlib.Path("E:\GolfData\Frames")

In [3]:
image_count = len(list(data_dir.glob('*/*.png')))
print(image_count)

12260


In [4]:
# loader parameters

batch_size = 1
img_height = 360
img_width = 640

In [5]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


Found 12260 files belonging to 2 classes.
Using 9808 files for training.


In [6]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


Found 12260 files belonging to 2 classes.
Using 2452 files for validation.


In [7]:
class_names = train_ds.class_names
print(class_names)

['Negatives', 'Positives']


In [8]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break


(1, 360, 640, 3)
(1,)


In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(200).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [13]:
num_classes = 2

model = Sequential([

    # preprocessing and feature augmentation
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)),
  layers.experimental.preprocessing.RandomRotation(0.1),
  layers.experimental.preprocessing.RandomZoom(0.1),

  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(64, activation='relu'),
  layers.Dense(num_classes)
])


In [14]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


In [15]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 360, 640, 3)       0         
_________________________________________________________________
random_flip_1 (RandomFlip)   (None, 360, 640, 3)       0         
_________________________________________________________________
random_rotation_1 (RandomRot (None, 360, 640, 3)       0         
_________________________________________________________________
random_zoom_1 (RandomZoom)   (None, 360, 640, 3)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 360, 640, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 180, 320, 16)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 180, 320, 32)     

In [16]:
epochs=5
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/5
1559/9808 [===>..........................] - ETA: 2:50 - loss: 0.7322 - accuracy: 0.5253

KeyboardInterrupt: 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [37]:
# prediction

# positieve predictie
image_path = "E:\GolfData\Tests\Positives\AAAAAAAA.png"
img = keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print("This image most likely belongs to {} with a {:.2f} percent confidence.".format(class_names[np.argmax(score)], 100 * np.max(score)))

# negatieve predictie
image_path = "E:\GolfData\Tests\\Negatives\\AAAAFishing-Hobby-And-A-Passion.png"
img = keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print("This image most likely belongs to {} with a {:.2f} percent confidence.".format(class_names[np.argmax(score)], 100 * np.max(score)))


This image most likely belongs to Negatives with a 51.49 percent confidence.
This image most likely belongs to Negatives with a 51.89 percent confidence.


In [40]:
# save model
model.save("./supervisedNeuralNet/models/model")


INFO:tensorflow:Assets written to: ./supervisedNeuralNet/models/model\assets
